<a href="https://colab.research.google.com/github/LeFalko/detect_CS/blob/master/Detect_CS_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
import os

#download U'n'Eye
!git clone https://github.com/berenslab/uneye.git

# remove folders and files that are not needed
os.chdir('uneye')
import shutil
files = ['README.md','UnEye.ipynb','logo.jpeg','requirements_lin.txt','requirements_mac.txt','requirements_wind.txt','setup.py']
for i,f in enumerate(files):
    try:
        os.remove(f)
    except:
        print("File already deleted: ", f)
          
folders = ['.git','analysis scripts','data']
for i,f in enumerate(folders) :
    try:
        shutil.rmtree(f)
    except:
        print("Folder already deleted: ", f)
os.makedirs('training',exist_ok = True)
os.makedirs('work',exist_ok = True)
os.makedirs('work/LabelYourData',exist_ok = True)
os.makedirs('work/Output',exist_ok = True)
os.makedirs('work/TrainYourNetwork',exist_ok = True)

# add a patch to use U'n'Eye as CS detector
!git clone https://github.com/LeFalko/detect_CS.git

# to do math operations
import numpy as np
import scipy.io as io

# to read .mat files
!pip install mat4py
import mat4py
# to do read .pkl files
import pandas as pd

# to do clustering
!pip install hdbscan

import uneye
!pip install umap-learn
from detect_CS import *
#to download files from the internet
import urllib

Cloning into 'uneye'...
remote: Enumerating objects: 990, done.
remote: Total 990 (delta 0), reused 0 (delta 0), pack-reused 990
Receiving objects: 100% (990/990), 169.38 MiB | 22.94 MiB/s, done.
Resolving deltas: 100% (211/211), done.
Checking out files: 100% (489/489), done.
Cloning into 'detect_CS'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 470 (delta 66), reused 56 (delta 23), pack-reused 351
Receiving objects: 100% (470/470), 2.11 MiB | 25.45 MiB/s, done.
Resolving deltas: 100% (273/273), done.


NameError: ignored

In [3]:
# Set all the following parameters before running the cell
field_LFP = ['RAW']         # in .mat files where the LFP data is in the variable lfp then set: field_LFP = ['lfp']
                            # in .mat files where the LFP data is in the structure Data.lfp then set: field_LFP = ['Data','lfp']
                            # in .pkl file where the LFP data is in the key ['lfp'] then set: field_LFP = ['lfp']
                            # you can use either the LFP signal or the wide-band signal
field_high_pass = ['HIGH'] # same comment as above
field_label = ['Labels'] # same comment as above

# set the name of the weights of the network
weights_name = 'my_weight_name' # the weights are going to be saved in the folder /training

# set sample frequency of your recording
sampfreq = 25000 # in Hz

# size of the convolutional and maxpooling operations
ks = 9 # convolution kernel size 
mp = 7 # maxpooling size
# the number of bins (nb) taken into account by the network is given by
# nb = mp**2+mp**2*ks+(mp*ks)-mp+2*ks-2 


########################################################################################################################################################################################
training_folder = '/gdrive/My Drive/uneye/work/TrainYourNetwork/'
training_files = os.listdir(training_folder)

bigLFP = []
bigHIGH = []
bigLabels = []
for i,tf in enumerate(training_files):
    LFP,High,Labels,_ = load_data(filename = training_folder+tf,field_LFP = field_LFP,field_high_pass = field_high_pass, field_label = field_label)
    interval_inspected = create_random_intervals(sampfreq,LFP,Labels)
    compLFP,compHIGH,compLabels = concatenate_segments(LFP,High,interval_inspected,Labels)
    bigLFP = np.concatenate((bigLFP,compLFP))
    bigHIGH = np.concatenate((bigHIGH,compHIGH))
    bigLabels = np.concatenate((bigLabels,compLabels))
val_samples = int(np.max((np.round(.1*len(bigLFP)/sampfreq),1)))
model = uneye.DNN(max_iter = 3000, ks=ks,mp=mp, weights_name=weights_name,
                  sampfreq = sampfreq,augmentation = False,val_samples = val_samples, doDiff = False)
model.train(bigLFP,bigHIGH,bigLabels)

TypeError: ignored